### Exercicio 1:


1.  Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:
    a. Cada reunião ocupa uma sala (enumeradas $$1...S\,$$) durante um “slot” $$(\text{tempo},\text{dia})$$.  Assume-se os dias enumerados $$1..D$$ e, em cada dia, os tempos enumerados $$1..T$$.
    b.  Cada reunião tem associado um projeto (enumerados $$1..P$$) e um conjunto de participantes. Os diferentes colaboradores são enumerados $$1..C$$.
    c. Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. São “inputs” do problema o conjunto de colaboradores de cada projeto, o seu líder e o número de reuniões semanais.
    d. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de  $$50\%$$ do total de colaboradores do projeto.  A disponibilidade de cada participante, incluindo o lider,  é um conjunto de “slots” (“inputs” do problema). 

## Inputs do Problema:

S       - Número de salas \
D       - Dias \
T       - Tempo \
P       - Número de projeto \
C       - Número de colaboradores \
(T,D)   - Formato de um slot (tempo,dia)

slots   - Conjunto de slots da semana

Sendo *colaboradores* o conjunto de todos os colaboradores, *colaboradores*<sub>**i**</sub> define todos os slots em que o colaborador está disponível.

Sendo *projetos* o conjunto de todos os projetos, *projetos*<sub>**i**</sub> define o número de reuniões semanais do projeto, a lista dos colaboradores do projeto e um colaborador do projeto como líder do mesmo.

## Restrições da construção do horário

1. O líder têm que participar em todas as reuniões do projeto
1. Todas as reuniões de um dado projeto têm de ter uma participação mínima de 50% dos colaboradores
1. Uma dada sala só pode ter uma e uma só reunião por slot
1. Um colaborador não pode participar em mais de uma reunião por slot
1. Todos os projetos tem um dado número de reuniões
1. Todos os colaboradores tem um conjunto de slots em que estão disponíveis
1. Um colaborador não pode participar numa reunião fora da sua disponibilidades
1. Um colaborador não pode participar numa reunião de um projeto ao qual não está alocado

In [1]:
%pip install ortools

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 973.6 kB/s eta 0:00:000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 2.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 KB 2.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 KB 2.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 KB 2.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 828.6 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [89]:
### Completar
from ortools.linear_solver import pywraplp
import random

random.seed()

S = 4
D = 5
T = 16
P = 1
C = 10

slots = []
colaboradores = {} # colaboradores[i] = [s] - [s] são slots em que o colaborador está disponível

projetos = {} # projetos[i] = (l,r,[c]) - l é o lider do projeto
              #                         - r é o número de reuniões do projeto                  
              #                         - [c] são os colaboradores associados ao projeto

for d in range(0,D):
    for t in range(0,T):
        slots.append((t+1,d+1))

#gerar exemplo
# 1: Disponibilidade dos Colaboradores
for c in range(1,C+1):
    disp = random.randrange(4,7)
    colaboradores[c] = random.choices(slots,k = disp)

# 2: Criação dos projetos
for p in range(1,P+1):
    r = random.randrange(1,3)
    nc = random.randrange(2,4)
    i = 0
    while i < nc:
        choice = random.choice([1,2,3,4,5,6,7,8,9,10])
        colabs = []
        if choice not in colabs:
            colabs.append(choice)
            i+=1
    lider = random.choice(colabs)
    projetos[p] = (lider,r,colabs)

alocaSalas = {} # Matriz que guarda se uma dada sala s está atribuida a um projeto p num dado slot (t,d)
alocaColab = {} # Matriz que guarda se um dado colaborador está atribuido a um projeto p num dado slot (t,d)

# inicializar alocaSalas e alocaColab

solver = pywraplp.Solver.CreateSolver('SCIP')

for s in range(1,S+1):
    alocaSalas[s] = {}
    for d in range(1,D+1):
        alocaSalas[s][d] = {}
        for t in range(1,T+1):
            alocaSalas[s][d][t] = {}
            for p in range(1,P+1):
                alocaSalas[s][d][t][p] = solver.BoolVar('alocaSalas[%i][%i][%i][%i]' % (s,d,t,p))

for c in range(1,C+1):
    alocaColab[c] = {}
    for d in range(1,D+1):
        alocaColab[c][d] = {}
        for t in range(1,T+1):
            alocaColab[c][d][t] = {}
            for p in range(1,P+1):
                alocaColab[c][d][t][p] = solver.BoolVar('alocaColab[%i][%i][%i][%i]' % (c,d,t,p))

# O líder têm que participar em todas as reuniões do projeto

for d in range(1,D+1):
    for t in range(1,T+1):
        for p in range(1,P+1):
            l = projetos[p][0]
            solver.Add(sum(alocaSalas[s][d][t][p] for s in range(1,S+1)) == alocaColab[l][d][t][p])

# Uma sala só pode ter uma reunião por slot

for s in range(1,S+1):
    for d in range(1,D+1):
        for t in range(1,T+1):
            solver.Add(sum(alocaSalas[s][d][t][p] for p in range(1,P+1)) <= 1)

# Um colaborador não pode ser alocado a uma reunião cujo slot está fora da disponibilidade do colaborador

for c in range(1,C+1):
    for p in range(1,P+1):
        for d in range(1,D+1):
            for t in range(1,T+1):
                if (t,d) not in colaboradores[c]:
                    solver.Add(alocaColab[c][d][t][p] == 0)

# Um colaborador não pode participar em mais de uma reunião no mesmo slot

for c in range(1,C+1):
    for d in range(1,D+1):
        for t in range(1,T+1):
            solver.Add(sum(alocaColab[c][d][t][p] for p in range(1,P+1)) <= 1)

# Um projeto tem que ter um dado número de reuniões

for p in range(1,P+1):
    solver.Add(sum(alocaSalas[s][d][t][p] for s in range(1,1+S) for d in range(1,D+1) for t in range(1,T+1)) == projetos[p][1])

# Uma reunião têm que ter no mínimo uma presença de 50% dos colaboradores

for p in range(1,P+1):
    for d in range(1,D+1):
        for t in range(1,T+1):
            l = projetos[p][0]
            solver.Add(sum(alocaColab[c][d][t][p] for c in range(1,C+1)) >= 0.5*len(projetos[p][2])*alocaColab[l][d][t][p])

# Um colaborador não pode participar numa reunião de um projeto ao qual não está alocado

for c in range(1,C+1):
    for p in range(1,P+1):
        if c not in projetos[p][2]:
            solver.Add(sum(alocaColab[c][d][t][p] for d in range(1,D+1) for t in range(1,T+1)) == 0)

# Solução para o problema
r = solver.Solve()

if r == pywraplp.Solver.OPTIMAL:
    print("Solução encontrada")
else:
    print("Solução não encontrada")


Solução encontrada
